In [2]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
        
    return str_text

In [3]:
# read_file('../UPDATED_NLP_COURSE/06-Deep-Learning/moby_dick_four_chapters.txt')

In [4]:
# read_file('../UPDATED_NLP_COURSE/06-Deep-Learning/melville-moby_dick.txt')

In [5]:
import spacy

In [6]:
nlp = spacy.load('en_core_web_lg', disable=['parser', 'tagger', 'ner'])

In [7]:
nlp.max_length = 1198623

In [8]:
def seperate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [9]:
d = read_file('../UPDATED_NLP_COURSE/06-Deep-Learning/moby_dick_four_chapters.txt')

In [10]:
tokens = seperate_punc(d)

In [11]:
len(tokens)

11394

In [12]:
# 25 words --> network predict #26

In [13]:
train_len = 25 + 1

text_sequences = []

for i in range(train_len, len(tokens)):
    seq = tokens[i - train_len:i]
    
    text_sequences.append(seq)

In [14]:
type(text_sequences)

list

In [15]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [16]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [17]:
' '.join(text_sequences[2])

'ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i'

In [18]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [19]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [20]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [21]:
# sequences[1]

In [22]:
for i in sequences[0]:
    print(f"{i} : {tokenizer.index_word[i]}")
# tokenizer.index_word

964 : call
14 : me
265 : ishmael
51 : some
263 : years
416 : ago
87 : never
222 : mind
129 : how
111 : long
962 : precisely
262 : having
50 : little
43 : or
37 : no
321 : money
7 : in
23 : my
555 : purse
3 : and
150 : nothing
261 : particular
6 : to
2704 : interest
14 : me
24 : on


In [23]:
# tokenizer.word_counts

In [24]:
vocabulary_size = len(tokenizer.word_counts)

In [25]:
vocabulary_size

2709

In [26]:
type(sequences)

list

In [27]:
import numpy as np

In [28]:
sequences = np.array(sequences)

In [29]:
sequences

array([[ 964,   14,  265, ..., 2704,   14,   24],
       [  14,  265,   51, ...,   14,   24,  965],
       [ 265,   51,  263, ...,   24,  965,    5],
       ...,
       [ 960,   12,  168, ...,  264,   53,    2],
       [  12,  168, 2703, ...,   53,    2, 2709],
       [ 168, 2703,    3, ...,    2, 2709,   26]])

In [30]:
from keras.utils import to_categorical

In [38]:
X = sequences[:,:-1]

In [34]:
y = sequences[:,-1]

In [36]:
y = to_categorical(y,num_classes=vocabulary_size+1)

In [39]:
seq_len = X.shape[1]

In [40]:
X.shape

(11368, 25)

In [41]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [43]:
def create_model(vocabulary_size, seq_len):
    
    model = Sequential()
    model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
    model.add(LSTM(seq_len * 2, return_sequences=True))
    model.add(LSTM(seq_len * 2))
    model.add(Dense(seq_len * 2, activation='relu'))
    
    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.summary()
    
    return model

In [47]:
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 25, 25)            67750     
_________________________________________________________________
lstm_3 (LSTM)                (None, 25, 50)            15200     
_________________________________________________________________
lstm_4 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 2710)              138210    
Total params: 243,910
Trainable params: 243,910
Non-trainable params: 0
_________________________________________________________________


In [45]:
from pickle import dump,load

In [87]:
model.fit(X, y, batch_size=128, epochs=50, verbose=1)

Epoch 1/50
11368/11368 [==============================] - 4s 359us/step - loss: 0.2021 - accuracy: 0.9624
Epoch 2/50
11368/11368 [==============================] - 4s 360us/step - loss: 0.1960 - accuracy: 0.9652
Epoch 3/50
11368/11368 [==============================] - 4s 362us/step - loss: 0.1903 - accuracy: 0.9661
Epoch 4/50
11368/11368 [==============================] - 4s 370us/step - loss: 0.1848 - accuracy: 0.9681
Epoch 5/50
11368/11368 [==============================] - 4s 366us/step - loss: 0.1835 - accuracy: 0.9682
Epoch 6/50
11368/11368 [==============================] - 4s 369us/step - loss: 0.1852 - accuracy: 0.9680
Epoch 7/50
11368/11368 [==============================] - 4s 369us/step - loss: 0.1902 - accuracy: 0.9656
Epoch 8/50
11368/11368 [==============================] - 4s 372us/step - loss: 0.1957 - accuracy: 0.9627
Epoch 9/50
11368/11368 [==============================] - 4s 373us/step - loss: 0.2051 - accuracy: 0.9599
Epoch 10/50
11368/11368 [=====================

In [89]:
model.save('my_mobydick_model.h5')

In [90]:
dump(tokenizer, open('my_simpletokenizer', 'wb'))

In [53]:
from keras.preprocessing.sequence import pad_sequences

In [63]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    
    output_text = []
    
    input_text = seed_text
    
    for i in range(num_gen_words):
        
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        pred_word_ind = model.predict_classes(pad_encoded, verbose = 0)[0]
        
        pred_word = tokenizer.index_word[pred_word_ind]
        
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
    
    return ' '.join(output_text)

In [55]:
text_sequences[0]

['call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on']

In [80]:
import random
random.seed(42)
random_pick = random.randint(0, len(text_sequences))

In [81]:
random_seed_text = text_sequences[random_pick]

In [82]:
random_seed_text

['predicament',
 'for',
 'though',
 'i',
 'tried',
 'to',
 'move',
 'his',
 'arm',
 'unlock',
 'his',
 'bridegroom',
 'clasp',
 'yet',
 'sleeping',
 'as',
 'he',
 'was',
 'he',
 'still',
 'hugged',
 'me',
 'tightly',
 'as',
 'though',
 'naught']

In [83]:
seed_text = ' '.join(random_seed_text)

In [84]:
seed_text

'predicament for though i tried to move his arm unlock his bridegroom clasp yet sleeping as he was he still hugged me tightly as though naught'

In [94]:
generate_text(model, tokenizer, seq_len, seed_text=seed_text, num_gen_words=25)

"was prepared again and my all the world oh yes devil captain ahab ' do were it not swerve me strove from thee orders ye"

In [66]:
from keras.models import load_model

In [91]:
model = load_model('../UPDATED_NLP_COURSE/06-Deep-Learning/epochBIG.h5')

In [93]:
tokenizer = load(open('../UPDATED_NLP_COURSE/06-Deep-Learning/epochBIG', 'rb'))